# Collection-1

## get nonAC and AC

In [14]:
import multiprocessing
from multiprocessing import Pool
from tqdm import tqdm
import numpy as np
import random
from rdkit import Chem
from random import randint
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AtomPairs import Pairs
import pandas as pd
from rdkit.Chem import DataStructs
from Levenshtein import distance as levenshtein

from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric as GraphFramework
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol

In [2]:

def scaffold_compute(mol):
    try:
        skeleton = GraphFramework(mol)      
    except Exception:  # In the very rare case this doesn't work, use a normal scaffold
        skeleton = GetScaffoldForMol(mol)
    skeleton_fp = AllChem.GetMorganFingerprintAsBitVect(skeleton, radius=2, nBits=1024)
    return skeleton_fp


def computesim_time(index_1, index_2):
    mol1 = Chem.MolFromSmiles(data_smiles[index_1])
    mol2 = Chem.MolFromSmiles(data_smiles[index_2])


    simi_levenshtein = 1-(levenshtein(data_smiles[index_1], data_smiles[index_2]) / max(len(data_smiles[index_1]), len(data_smiles[index_2])))
    # tanimoto similarity
    mol1_morganFP = AllChem.GetMorganFingerprintAsBitVect(mol1, radius=2, nBits=1024)
    mol2_morganFP = AllChem.GetMorganFingerprintAsBitVect(mol2, radius=2, nBits=1024)
    simi_tanimoto = DataStructs.TanimotoSimilarity(mol1_morganFP, mol2_morganFP)
    # scaffold similarity
    mol1_scaFP = scaffold_compute(mol1)
    mol2_scaFP = scaffold_compute(mol2)
    simi_scaffold = DataStructs.TanimotoSimilarity(mol1_scaFP, mol2_scaFP)

    # similarity
    m_tani = simi_tanimoto >= 0.9
    m_scaff = simi_scaffold >= 0.9
    m_leve = simi_levenshtein >= 0.9
    simi = int(m_tani + m_scaff + m_leve)
    single_reult = (data_smiles[index_1], data_smiles[index_2], bioactivity[index_1], bioactivity[index_2], simi, data_ID[index_1], data_ID[index_2])
    return single_reult


## 30 datasets for activity cliff detection

In [4]:
# read data
data_1 = pd.read_csv('/home/xxx/anaconda3/envs/ACM/lib/python3.8/site-packages/MoleculeACE/Data/benchmark_data/CHEMBL3979_EC50.csv')
data_1['ID'] = data_1.index

data_smiles = data_1.loc[data_1['split']=='train', 'smiles'].tolist()
bioactivity = data_1.loc[data_1['split']=='train', 'y'].tolist()
bioactivity = np.array(bioactivity) + 9



data_ID = data_1['ID'].tolist()

pair_list = []
for j in range(len(data_smiles)):
    for k in range(j+1, len(data_smiles)):
        pair_list.append((j, k))

def compute_simi(a):
    all_result = []
    index_1 = a[0]
    index_2 = a[1]
    single_reult = computesim_time(index_1, index_2)
    if single_reult[4] != 0:
        return single_reult

with Pool(20)as proc:
    results = list(
        tqdm(
             proc.imap_unordered(compute_simi, pair_list,
                     ),
            total=len(pair_list)
        ))


smile_1, smile_2, ki_1, ki_2, is_Simi, smile1_ID, smile2_ID = [], [], [], [], [], [], []
for i in results:
    try:
        smile_1.append(i[0])
        smile_2.append(i[1])
        ki_1.append(i[2])
        ki_2.append(i[3])
        is_Simi.append(i[4])
        smile1_ID.append(i[5])
        smile2_ID.append(i[6])

    except:
        pass

df = pd.DataFrame({
    'smile_1': smile_1,
    'smile_2': smile_2,
    'ki_1': ki_1,
    'ki_2': ki_2,
    'is_Simi': is_Simi,
    'smile1_ID': smile1_ID,
    'smile2_ID': smile2_ID
})

df.to_csv('./collection-1/cm3979_AC_0918.csv', index=False)

100%|██████████| 404550/404550 [00:41<00:00, 9664.77it/s] 


In [5]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

#读取数据
data_frame = pd.read_csv('./collection/cm3979_AC_0918.csv')
data_frame.tail(3)

,smile_1,smile_2,ki_1,ki_2,is_Simi,smile1_ID,smile2_ID
1795,CC(C)c1onc(-c2ccccc2Cl)c1COc1ccc(COc2ccc(CC(=O...,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(CNc2ccc(C...,8.397940,6.045757,1,895,896
1796,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(CNc2ccc(C...,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(CNc2ccc(C...,6.045757,6.096910,1,896,899
1797,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(COc2ccc(C...,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(CNc2ccc(C...,4.958607,6.096910,2,898,899


### transfer smile_1 and smile_2 to get the second dataframe, then merge it with the original dataframe

In [6]:
smi1_new = data_frame['smile_2']
smi2_new = data_frame['smile_1']
ki1_new = data_frame['ki_2']
ki2_new = data_frame['ki_1']
smi1ID_new = data_frame['smile2_ID']
smi2ID_new = data_frame['smile1_ID']
isSimi_new = data_frame['is_Simi']

data_frame2 = pd.DataFrame({
    'smile_1': smi1_new,
    'smile_2': smi2_new,
    'ki_1': ki1_new,
    'ki_2': ki2_new,
    'is_Simi': isSimi_new,
    'smile1_ID': smi1ID_new,
    'smile2_ID': smi2ID_new
})

data_frame = pd.concat([data_frame, data_frame2], ignore_index=True)
data_frame.tail(3)

,smile_1,smile_2,ki_1,ki_2,is_Simi,smile1_ID,smile2_ID
3593,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(CNc2ccc(C...,CC(C)c1onc(-c2ccccc2Cl)c1COc1ccc(COc2ccc(CC(=O...,6.045757,8.397940,1,896,895
3594,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(CNc2ccc(C...,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(CNc2ccc(C...,6.096910,6.045757,1,899,896
3595,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(CNc2ccc(C...,CC(C)c1onc(-c2c(Cl)cccc2Cl)c1COc1ccc(COc2ccc(C...,6.096910,4.958607,2,899,898


## identifing AC and nonAC

In [7]:
ki_1 = data_frame['ki_1'].tolist()
ki_2 = data_frame['ki_2'].tolist()

data_ki = []
for i in range(len(ki_1)):
    pair_ki = (ki_1[i], ki_2[i])
    sorted_pairki = tuple(sorted(pair_ki, reverse=True))
    data_ki.append(sorted_pairki)

is_AC = []
for a in data_ki:
    if a[0] - a[1] <= 1:
        is_AC.append(0)
    else:
        is_AC.append(1)

data_frame['is_AC'] = is_AC
count = data_frame['is_AC'].value_counts()

AC(1)和nonAC(0)的数量: 
 0    2606
1     990
Name: is_AC, dtype: int64


In [8]:
grouped = data_frame.groupby('smile1_ID').groups
group_smiindex = [list(grouped[key]) for key in grouped.keys()]
print(group_smiindex[1])

group_Ki = []
for i in group_smiindex:
    group_Ki.append([data_frame['is_AC'][j] for j in i])
print(group_Ki[1])

[6, 7, 8, 1798]
[0, 0, 0, 0]


In [9]:
import math
group_smi1 = []
group_smi1_ki = []
group_smi1num = []
pki_anchor = []

for i in group_smiindex:
    group_smi1.append(data_frame['smile_1'][i[0]])
    group_smi1num.append(data_frame['smile1_ID'][i[0]])
    group_smi1_ki.append(data_frame['ki_1'][i[0]])

In [10]:
group_smi2 = []
group_smi2_pki = []
for i in group_smiindex:
    group_smi2.append([data_frame['smile_2'][j] for j in i])
    group_smi2_pki.append([data_frame['ki_2'][j] for j in i])

delta_pki = []
for i in group_smiindex:
    pki_smi2 = [data_frame['ki_2'][j] for j in i]
    delta_pki.append([abs(-x + data_frame['ki_1'][i[0]]) for x in pki_smi2])


anchor的ID: 5
anchor的SMILES: CCCC(Cc1ccc(OC)c(C(=O)NCc2ccc(C(F)(F)F)cc2)c1)C(=O)O
与anchor相似的smiles: ['CCC(Cc1ccc(OC)c(C(=O)NCc2ccc(C(F)(F)F)cc2F)c1)C(=O)O', 'CCCC(Cc1ccc(OC)c(CNC(=O)c2ccc(C(F)(F)F)cc2F)c1)C(=O)O', 'CCCCC(Cc1ccc(OC)c(CNC(=O)c2ccc(C(F)(F)F)cc2)c1)C(=O)O', 'CCC(Cc1ccc(OC)c(C(=O)NCCc2ccc(C(F)(F)F)cc2)c1)C(=O)O', 'CCC(Cc1ccc(OC)c(C(=O)NCc2ccc(OC(F)(F)F)cc2)c1)C(=O)O', 'CCOC(Cc1ccc(OC)c(C(=O)NCc2ccc(C(F)(F)F)cc2)c1)C(=O)O']
这些SMILES与anchor的pki差值: [0.3802112417116099, 1.77815125038365, 0.5351132016973503, 0.20411998265593034, 0.42596873227229004, 0.09691001300805002]


In [11]:
save_SI = []
for j in range(len(group_Ki)):
    d = {}
    d['anchor_smi'] = group_smi1[j]
    d['delta_nonAC'] = []
    d['delta_AC'] = []
    d['smile_nonAC'] = []
    d['smile_AC'] = []
    d['nonAC_pki'] = []
    d['AC_pki'] = []

    for i, x in enumerate(delta_pki[j]):
        if x <= 1:
            d['delta_nonAC'].append(x)
            d['smile_nonAC'].append(group_smi2[j][i])
            d['nonAC_pki'].append(group_smi2_pki[j][i])
        else:
            d['delta_AC'].append(x)
            d['smile_AC'].append(group_smi2[j][i])
            d['AC_pki'].append(group_smi2_pki[j][i])
    save_SI.append(d)
print(save_SI[1])

{'anchor_smi': 'CCC(Cc1ccc(OC)c(C(=O)NCc2ccc(OC(F)(F)F)cc2)c1)C(=O)O', 'delta_nonAC': [0.5228787452803401, 0.42596873227229004, 0.045757490560680125, 0.2218487496163597], 'delta_AC': [], 'smile_nonAC': ['CCOC(Cc1ccc(OC)c(C(=O)NCc2ccc(C(F)(F)F)cc2)c1)C(=O)O', 'CCCC(Cc1ccc(OC)c(C(=O)NCc2ccc(C(F)(F)F)cc2)c1)C(=O)O', 'CCC(Cc1ccc(OC)c(C(=O)NCc2ccc(C(F)(F)F)cc2F)c1)C(=O)O', 'CCC(Cc1ccc(OC)c(C(=O)NCCc2ccc(C(F)(F)F)cc2)c1)C(=O)O'], 'smile_AC': [], 'nonAC_pki': [5.52287874528034, 5.61978875828839, 6.0, 5.82390874094432], 'AC_pki': []}


### save as binary format file

In [12]:
df = pd.DataFrame(save_SI)
df['anchor_ki'] = group_smi1_ki
df.columns = df.columns.map(str)
# Save DataFrame to Parquet file
df.to_parquet('./collection-1/cm3979_AC_0918.parquet')

In [70]:
import pyarrow as pa
import pyarrow.parquet as pq
df_0715 = pq.read_table('./collection/cm239_AC_0918.parquet').to_pandas()
# df_0715.tail(5)